In [1]:
import pandas as pd
import mysql.connector

## Connect to the Database

In [2]:
connection = mysql.connector.connect(host = 'localhost', user = 'root', password = 'root', port = 3306, database='testdb')
cursor = connection.cursor(buffered=True)
cursor.execute("SELECT VERSION()")
cursor.fetchone()

('8.3.0',)

## Create Table Account Profile

In [3]:
# create account_prifile table
cursor.execute('DROP TABLE IF EXISTS account_profile;')
cursor.execute('''
               CREATE TABLE account_profile(
                   customer_id BIGINT PRIMARY KEY,
                   card_avg decimal(10,2),
                   check_avg decimal(10,2),
                   deposit_avg decimal(10,2),
                   `loan payment_avg` decimal(10,2),
                   transfer_avg decimal(10,2),
                   withdrawal_avg decimal(10,2),
                   card_count int,
                   check_count int,
                   deposit_count int,
                   `loan payment_count` int,
                   transfer_count int,
                   wihdrawal_count int
                   );
               ''')

## Create Stored Procedure

In [4]:
# create stored procedure insert into account_profile if not exists or update if exists
cursor.execute('DROP PROCEDURE IF EXISTS account_Profile;')
cursor.execute('''
CREATE PROCEDURE account_Profile()
    BEGIN
        INSERT INTO account_profile
            select
                customer_id,
                ROUND(AVG(case when transaction_type = 'Card' then amount end), 2) as card_avg,
                ROUND(AVG(case when transaction_type = 'Check' then amount end), 2) as check_avg,
                ROUND(AVG(case when transaction_type = 'Deposit' then amount end), 2) as Deposit_avg,
                ROUND(AVG(case when transaction_type = 'Loan Payment' then amount end), 2) as `Loan Payment_avg`,
                ROUND(AVG(case when transaction_type = 'Transfer' then amount end), 2) as Transfer_avg,
                ROUND(AVG(case when transaction_type = 'Withdrawal' then amount end), 2) as Withdrawal_avg,
                SUM(case when transaction_type = 'Card' then 1 else 0 end)as card_count,
                SUM(case when transaction_type = 'Check' then 1 else 0 end)as check_count,
                SUM(case when transaction_type = 'Deposit' then 1 else 0 end)as Deposit_count,
                SUM(case when transaction_type = 'Loan Payment' then 1 else 0 end)as `Loan Payment_count`,
                SUM(case when transaction_type = 'Transfer' then 1 else 0 end)as Transfer_count,
                SUM(case when transaction_type = 'Withdrawal' then 1 else 0 end)as Withdrawal_count
            from transaction
            group by customer_id
            order by customer_id
        ON DUPLICATE KEY UPDATE
            card_avg = values(card_avg),
            check_avg = values(check_avg),
            deposit_avg = values(deposit_avg),
            `loan payment_avg` = values(`loan payment_avg`),
            transfer_avg = values(transfer_avg),
            withdrawal_avg = values(withdrawal_avg),
            card_count = values(card_count),
            check_count = values(check_count),
            deposit_count = values(deposit_count),
            `loan payment_count` = values(`loan payment_count`),
            transfer_count = values(transfer_count),
            wihdrawal_count = values(wihdrawal_count);
    END;
''')

In [5]:
# Test the stored procedure
cursor.execute('CALL account_profile();')
connection.commit()


In [6]:
# Top 5 rows of account_profile
cursor.execute('select * from account_profile order by customer_id limit 5;')
profile = pd.DataFrame(cursor.fetchall(), columns = [x[0] for x in cursor.description])
profile

,customer_id,card_avg,check_avg,deposit_avg,loan payment_avg,transfer_avg,withdrawal_avg,card_count,check_count,deposit_count,loan payment_count,transfer_count,wihdrawal_count
0,1000,26.47,4378.71,28869.12,12108.41,1524.85,694.78,1,2,2,2,1,3
1,1001,51.53,4237.00,None,28348.91,2231.76,540.01,4,2,0,3,4,4
2,1002,None,5117.56,27592.44,22833.47,1642.22,692.91,0,1,3,3,2,1
3,1003,None,5474.51,None,None,1554.03,1067.27,0,2,0,0,1,1
4,1004,38.21,None,23236.59,14425.75,2031.13,1032.39,6,0,3,3,1,2


In [7]:
# Bottom 5 rows of account_profile
cursor.execute('select * from account_profile order by customer_id desc limit 5;')
profile = pd.DataFrame(cursor.fetchall(), columns = [x[0] for x in cursor.description])
profile

,customer_id,card_avg,check_avg,deposit_avg,loan payment_avg,transfer_avg,withdrawal_avg,card_count,check_count,deposit_count,loan payment_count,transfer_count,wihdrawal_count
0,10999,37.89,11104.57,48534.17,18573.65,3316.20,757.11,2,2,2,2,1,2
1,10998,50.77,5206.95,27874.57,23206.13,3029.61,None,3,1,2,1,1,0
2,10997,84.71,5107.67,40858.35,25356.89,1284.74,None,1,1,5,1,4,0
3,10996,105.45,None,46180.04,29398.99,1804.69,407.88,2,0,1,3,1,1
4,10995,33.20,None,None,14913.75,2468.82,587.00,1,0,0,1,3,3


## Create Trigger

In [8]:
# after insert trigger to update account_profile table
cursor.execute('DROP TRIGGER IF EXISTS account_profile_update;')
# grant trigger permission
cursor.execute('''
    CREATE TRIGGER account_profile_update
    AFTER INSERT ON transaction
    FOR EACH ROW
    BEGIN
        CALL account_profile();
    END;
''')

In [9]:
cursor.execute('show triggers;')
cursor.fetchall()

[('account_profile_update',
  'INSERT',
  'transaction',
  'BEGIN\n        CALL account_profile();\n    END',
  'AFTER',
  datetime.datetime(2024, 4, 8, 20, 45, 1, 590000),
  {'ERROR_FOR_DIVISION_BY_ZERO',
   'NO_ENGINE_SUBSTITUTION',
   'NO_ZERO_DATE',
   'NO_ZERO_IN_DATE',
   'ONLY_FULL_GROUP_BY',
   'STRICT_TRANS_TABLES'},
  'root@%',
  'utf8mb4',
  'utf8mb4_0900_ai_ci',
  'utf8mb4_0900_ai_ci')]

## Test the Trigger

In [9]:
# unit test
# before insert
cursor.execute('select * from account_profile where customer_id = 1000;')
profile = pd.DataFrame(cursor.fetchall(), columns = [i[0] for i in cursor.description])
profile

,customer_id,card_avg,check_avg,deposit_avg,loan payment_avg,transfer_avg,withdrawal_avg,card_count,check_count,deposit_count,loan payment_count,transfer_count,wihdrawal_count
0,1000,29.70,3323.04,53392.84,22722.13,None,793.09,1,2,1,1,0,2


In [10]:
cursor.execute('select * from transaction where customer_id = 1000;')
cursor.fetchall()

[(3758, 1000, '2023-12-18 11:04:29', 553.4760387019085, 'Withdrawal'),
 (13622, 1000, '2023-12-16 17:23:23', 53392.83933905362, 'Deposit'),
 (28977, 1000, '2023-11-03 09:44:31', 3368.5231277126986, 'Check'),
 (45694, 1000, '2023-12-01 15:08:44', 3277.5548936338773, 'Check'),
 (62684, 1000, '2023-12-19 12:57:10', 29.70360265633457, 'Card'),
 (66420, 1000, '2023-09-09 12:10:16', 1032.705443722297, 'Withdrawal'),
 (66572, 1000, '2023-10-17 13:30:17', 22722.128700152007, 'Loan Payment')]

In [11]:
# insert new record for customer_id = 1000
cursor.execute(f'''
    INSERT INTO transaction
    VALUES (100000 ,1000, '2024-01-01 00:00:00', 100.00, 'Card');
''')
cursor.execute('select * from transaction where customer_id = 1000;')
cursor.fetchall()

[(3758, 1000, '2023-12-18 11:04:29', 553.4760387019085, 'Withdrawal'),
 (13622, 1000, '2023-12-16 17:23:23', 53392.83933905362, 'Deposit'),
 (28977, 1000, '2023-11-03 09:44:31', 3368.5231277126986, 'Check'),
 (45694, 1000, '2023-12-01 15:08:44', 3277.5548936338773, 'Check'),
 (62684, 1000, '2023-12-19 12:57:10', 29.70360265633457, 'Card'),
 (66420, 1000, '2023-09-09 12:10:16', 1032.705443722297, 'Withdrawal'),
 (66572, 1000, '2023-10-17 13:30:17', 22722.128700152007, 'Loan Payment'),
 (100000, 1000, '2024-01-01 00:00:00', 100.0, 'Card')]

In [12]:

cursor.execute('select * from account_profile where customer_id = 1000;')
pd.DataFrame(cursor.fetchall(), columns = [i[0] for i in cursor.description])

,customer_id,card_avg,check_avg,deposit_avg,loan payment_avg,transfer_avg,withdrawal_avg,card_count,check_count,deposit_count,loan payment_count,transfer_count,wihdrawal_count
0,1000,64.85,3323.04,53392.84,22722.13,None,793.09,2,2,1,1,0,2


In [13]:
# remove the test data
cursor.execute('delete from transaction where customer_id = 1000 and timestamp = "2024-01-01 00:00:00";')
cursor.execute('select * from transaction where customer_id = 1000;')
cursor.fetchall()

[(3758, 1000, '2023-12-18 11:04:29', 553.4760387019085, 'Withdrawal'),
 (13622, 1000, '2023-12-16 17:23:23', 53392.83933905362, 'Deposit'),
 (28977, 1000, '2023-11-03 09:44:31', 3368.5231277126986, 'Check'),
 (45694, 1000, '2023-12-01 15:08:44', 3277.5548936338773, 'Check'),
 (62684, 1000, '2023-12-19 12:57:10', 29.70360265633457, 'Card'),
 (66420, 1000, '2023-09-09 12:10:16', 1032.705443722297, 'Withdrawal'),
 (66572, 1000, '2023-10-17 13:30:17', 22722.128700152007, 'Loan Payment')]

In [14]:
cursor.execute('CALL account_profile();')
cursor.execute('select * from account_profile where customer_id = 1000;')
profile = pd.DataFrame(cursor.fetchall(), columns = [i[0] for i in cursor.description])
profile

,customer_id,card_avg,check_avg,deposit_avg,loan payment_avg,transfer_avg,withdrawal_avg,card_count,check_count,deposit_count,loan payment_count,transfer_count,wihdrawal_count
0,1000,29.70,3323.04,53392.84,22722.13,None,793.09,1,2,1,1,0,2


## Close the Connection

In [10]:
connection.commit()
cursor.close()
connection.close()